In [15]:
// Parse Input
using Newtonsoft.Json;
using System.Numerics;

/*
var input = System.IO.File.ReadAllLines("sample-input.txt");
/*/ 
var input = System.IO.File.ReadAllLines("input.txt");
//*/


In [16]:
static List<object> ParseToObject(string array){
    var stackOfObjects = new Stack<List<object>>();
    var chars = new List<char>();
    foreach(var p in array){
        switch(p){
            case '[':
                stackOfObjects.Push(new ());
                break;
            case ',':
                if(chars.Any()){
                    stackOfObjects.Peek().Add(int.Parse(new string(chars.ToArray())));
                    chars = new();
                }
                continue;
            case ']':
                var popped = stackOfObjects.Pop();

                if(chars.Any()){
                    popped.Add(int.Parse(new string(chars.ToArray())));
                    chars = new();
                }

                if(stackOfObjects.Any() == false){
                    return popped;
                }

                stackOfObjects.Peek().Add(popped);
            break;
            default:
                chars.Add(p);
            break;
        }
    }

    return null;
}


## Part 1

In [ ]:

IEnumerable<List<string>> GetGroups(string[] packets){
    var section = new List<string>();

    foreach(var line in packets){
        if(string.IsNullOrEmpty(line)){
            yield return section;
            section = new List<string>();
            continue;
        }
        section.Add(line);
    }

    if(section.Any()) yield return section;
}

var pairs = GetGroups(input).ToList();

In [ ]:

static bool? Compare(List<object> left, List<object> right){  
    for(var i = 0; i < left.Count; i++){
        if(i >= right.Count) {
            //Console.Write("Right out of items, ");
            return false;
        }
        
        var leftObj = left[i];
        var rightObj = right[i];

        //Console.WriteLine($"Comparing {string.Format(JsonConvert.SerializeObject(leftObj))} vs {JsonConvert.SerializeObject(rightObj)}");

        var innerOrder = (leftObj is List<object>, rightObj is List<object>) switch {
            (true, true) => Compare((List<object>)leftObj, (List<object>)rightObj),
            (true, false) => Compare((List<object>)leftObj, new List<object>(){rightObj}),
            (false, true) => Compare(new List<object>(){leftObj}, (List<object>)rightObj),
            (false, false) => CompareInt((int)leftObj, (int)rightObj)
        };

        if(innerOrder != null) return innerOrder;
    }

    if(left.Count < right.Count){
        //Console.Write("Left out of items, ");
        return true;
    }

    return null;
}

static bool? CompareInt(int left, int right){
    if(left == right) return null;

    if(left < right){    
        //Console.Write("Left is smaller, ");
        return true;
    }else{
        //Console.Write("Right is smaller, ");
        return false;
    }
}

In [ ]:
var orderedPairs = new List<int>();

for(var i = 0; i < pairs.Count; i++){
    var pair = pairs[i];
    var left = ParseToObject(pair[0]);
    var right = ParseToObject(pair[1]);

    //Console.WriteLine(JsonConvert.SerializeObject(new{left,right}));

    var result = Compare(left,right);
    if(result ?? false) orderedPairs.Add(i+1);
    //Console.WriteLine($"Pair {i+1} is {(result ?? false ? "right order" : "not ordered" )}");
}

orderedPairs

5507 - too high
5149 - too high

In [ ]:
orderedPairs.Sum()

## Part 2

In [17]:
public class PacketComparer : IComparer {
    int IComparer.Compare(Object x, Object y){
        var left = x as List<object>;
        var right = y as List<object>;

        for(var i = 0; i < left.Count; i++){
            if(i >= right.Count) {
                return 1;
            }
            
            var leftObj = left[i];
            var rightObj = right[i];

            var innerOrder = (leftObj is List<object>, rightObj is List<object>) switch {
                (true, true) => (new PacketComparer() as IComparer).Compare(leftObj, rightObj),
                (true, false) => (new PacketComparer() as IComparer).Compare(leftObj, new List<object>(){rightObj}),
                (false, true) => (new PacketComparer() as IComparer).Compare(new List<object>(){leftObj}, rightObj),
                (false, false) => (int)leftObj - (int)rightObj
            };

            if(innerOrder != 0) return innerOrder;
        }

        if(left.Count < right.Count){
            return -1;
        }

        return 0;    
    }
}

In [18]:
var dividers = new[]{"[[2]]", "[[6]]"};
var packets = input
    .Where(x => string.IsNullOrEmpty(x) == false)
    .Union(dividers)
    .Select(x => ParseToObject(x))
    .ToArray();
Array.Sort(packets, new PacketComparer());

In [19]:
var dividerPositions = new List<int>();

for(var i = 0; i < packets.Count(); i++){
    var packetsAsString = JsonConvert.SerializeObject(packets[i]);
    //Console.WriteLine(packetsAsString);
    if(dividers.Contains(packetsAsString)){
        dividerPositions.Add(i+1);
    }
}

dividerPositions.Aggregate((x,y) => x * y)

25038